# Verificação formal de programas:  *havoc* e *unfold* de ciclos


Na aula anterior vimos que uma abordagem à verificação de programa com ciclos, assenta no conceito de *invariante* de ciclo e nas suas propriedades de "inicialização", "preservação" e "utilidade".


Nesta aula vamos ver duas metodologias alternativas em que o programa original (com ciclos) é representado na linguagem intermédia de fluxos (sem ciclos), substituindo cada ciclo ${\sf while} \; b \;{\sf do } \;C$ por um fluxo $H$ tal que:
se $\{\phi\} H \{\psi\}$ se verificar, então $\{\phi\} {\sf while} \; b \;{\sf do } \;C \{\psi\}$ verifica-se também.
Uma das metedologias usa um novo comando `havoc x` e a outra vai desenrolando os ciclos um determinado número de vezes.

## *Havoc*

O comando `havoc x` pode ser descrito informalmente como uma atribuição a `x` de um valor arbitrário.  Em termos de denotação lógica usando a denotação WPC teremos 

$$
[{\sf havoc}\; x \; ; C] = \forall x. \,[C]
$$

Na metodologia *havoc*, o ciclo (${\sf while} \; b \;{\sf do }\{\theta\} \;C$), com anotação de invariante $\theta$ é transformado num fluxo não iterativo da seguinte forma

$$
{{\sf assert}\; \theta\; ; \sf havoc }\;\vec{x} \; ; (\,({\sf assume }\; b \wedge \theta \; ; \; C \; ; {\sf assert}\;\theta \; ; {\sf assume}\; \mathit{False}) \: || \:
{\sf assume}\; \neg b \wedge \theta \,)
$$

onde $\vec{x}$ representa as *variáveis atribuídas em $C$*.

Observe como a denotação do triplo de Hoare $\{\phi\} {\sf while} \; b \;{\sf do}\{\theta\}\,C \,\{\psi\}$,  traduzido desta forma,
permite garantir as propriedades de "inicialização", "preservação" e "utilidade" do invariante $\theta$

$$
\begin{array}{l}
[\,{\sf assume}\;\phi\; ;{{\sf assert}\; \theta\; ; \sf havoc }\;\vec{x} \; ; (\,({\sf assume }\; b \wedge \theta \; ; \; C \; ; {\sf assert}\;\theta \; ; {\sf assume}\; \mathit{False}) \: || \:
{\sf assume}\; \neg b \wedge \theta \,)\; ; {\sf assert} \; \psi \,] \\ = \\
\phi \to \theta \wedge \forall \vec{x}. \, (\,(b \wedge \theta \to [C\;; {\sf assert}\; \theta ]) \wedge (\neg b \wedge \theta \to \psi )\,)
\end{array}
$$

Note que $[ \,{\sf assume}\; \mathit{False}\;; {\sf assert} \; \psi \,] = \mathit{False} \to \psi = \mathit{True}$.



Como de costume, começamos por instalar o Z3 e declarar a função `prove`.

In [1]:
!pip install z3-solver

In [2]:
from z3 import *

def prove(f):
    s = Solver()
    s.add(Not(f))
    r = s.check()
    if r == unsat:
        print("Proved")
    else:
        print("Failed to prove")
        m = s.model()
        for v in m:
            print(v,'=', m[v])

Considere, por exemplo, o seguinte programa anotado (incluindo o invariante de ciclo) que calcula o produto de dois inteiros.

```python
assume n > 0;
x = 0;
i = 0;
while (i<n):
    invariante x == i*a and i <= n
    x = x+a;
    i = i+1;
assert x == n*a   
```
Para provar que este programa é correcto pelo método *havoc*, teremos que, em primeiro lugar,
proceder à sua tradução para a linguagem de fluxos com *havoc*. Uma vez que `x`e `i` são as únicas variáveis atribuidas no ciclo, a tradução dá origem ao seguinte programa, onde `inv = x == i*a and i <= n`:

```python
assume n>0;
x = 0; i = 0; 
assert inv;
havoc x; havoc i;
((assume i<n and inv; x=x+a; i=i+1; assert inv; assume False) || assume not(i<n) and inv);
assert x == n*a
```

Em seguida calcula-se a denotação lógica deste programa de fluxos (a sua VC) pela WPC.


A WPC deste programa resulta na seguinte fórmula lógica, onde ${\sf Ciclo}$ e ${\sf Corpo}$ representam, rspectivamente, as 3 últimas e as últimas 2 linhas deste programa de fluxos,  
${\sf inv} = x = i*a \wedge i \leq n$,  ${\sf pre} =n>0$ e  ${\sf pos}= x=n*a$.

$$
\begin{array}{l}
{\sf pre} \to ({\sf inv} \wedge [{\sf Ciclo}])[0/i][0/x] \\
\equiv \quad \mbox{(havoc)}\\
{\sf pre} \to ({\sf inv} \wedge \forall x.\forall i. [{\sf Corpo}])[0/i][0/x] \\
\equiv \quad \mbox{(porque $i$ e $x$ estão quantificadas)}\\
{\sf pre} \to {\sf inv}[0/i][0/x] \wedge \forall x.\forall i. [{\sf Corpo}] \\
\equiv \quad \mbox{(porque assume $\mathit{False}$ dá origem a $\mathit{False} \to \ldots = \mathit{True}$)} \\
{\sf pre} \to {\sf inv}[0/i][0/x] \;\wedge \; (\,\forall x.\forall i. \, (i<n \wedge {\sf inv} \to {\sf inv}[(i+1)/i][(x+a)/x]) \,\wedge\, (\neg(i<n) \wedge {\sf inv} \to {\sf pos}) \,)
\end{array}
$$

Observe a acção da quantificação das variáveis atribuidas no corpo do ciclo, ao fazer com que as atribuições `x=0; i=0` não tenham efeito no corpo do ciclo.





### Exercício 1

Complete a prova de correção do programa acima, usando o Z3 para provar a VC que gerou.
Note ao utilizar a função `substitute` que vai ter que usar `IntVal(0)` para criar a constante `0` como expressão Z3.

In [ ]:
n, x, i, a = Ints('n x i a')
pre = n>0
pos = x==n*a
inv = And(x==i*a,i<=n)
invS = substitute(substitute(inv,(i,IntVal(0))),(x,IntVal(0)))
impEsq = Implies(And(i<n,inv),substitute(substitute(inv,(i,i+1)),(x,x+a)))
impDir = Implies(And(Not(i<n), inv),pos)

vc= Implies(pre, And(invS, ForAll([x,i], And(impEsq,impDir))))

prove(vc)

Proved


## *Unfold* de ciclos

Uma outra metodologia (chamada *bounded model checking of software*) passa por simular a execução do ciclo, ${\sf while} \; b \;{\sf do } \;C$, um determinado número de vezes. 
Consiste basicamente em desenrolar os ciclos um certo número de vezes ($k$):

$$
\begin{array}{ll|ll}
{\sf if} \; b \;{\sf then } \;C; & \quad & \quad & {\sf if} \; b \;{\sf then } \;C; \\
\quad {\sf if} \; b \;{\sf then } \;C; &\quad & \quad & \quad {\sf if} \; b \;{\sf then } \;C; \\
\qquad \ldots &\quad & \quad & \qquad \ldots \\
\quad \qquad {\sf if} \; b \;{\sf then } \;\{C\;; {\sf assert}\; \neg b \} &\quad & \quad & \quad \qquad {\sf if} \; b \;{\sf then } \;\{C\;; {\sf assume}\; \neg b \}
\end{array}
$$
Para garantir a correcção (resp. a completude) desta abordagem, é inserida imediatamente após o desenrolamento do ciclo uma *unwinding assertion* (resp. uma *unwinding assumption*).
 
A *unwinding assertion* é usada para verificar se existem execuções que exigem mais do que $k$ iterações - se não existirem, isso significa que o programa original apenas possui execuções limitadas de comprimento $\leq k$, e a abordagem é, portanto, correcta.
Se o programa tiver execuções ilimitadas ou se não for prático verificar execuções limitadas devido ao seu comprimento, a técnica poderá ainda ser aplicada, mas não será correcta.
Nesse caso, é necessário, para garantir completude (garantir que não há falsos contra-exemplos) adicionar a assunção da condição do ciclo negada (denominada *unwinding assumption*) após a expansão do ciclo, o que excluirá as execuções que exigem mais de $k$ iterações de serem consideradas para verificação.

Após o *unfold* do ciclo, temos um programa sem ciclos cuja denotação lógica (a sua VC) pode ser obtida por qualquer das técnicas apresentadas na aula anterior (transformação do programa na linguagem intermédia de fluxo, seguida da geração da VC com WPC ou SPC).

### Exercício 2

Considere o seguinte programa:
```python
assume n > 0 and n < 4;
x = 1; 
r = 1; 
i = 1;
while (i < n):
    x = x+2;
    r = r+x;
    i = i+1
assert r == n*n;    
```

Aplique a metedologia do *unfold* de ciclos para provar a correcção deste programa usando a abordagem SPC. Para isso: 

*a)* Comece por desenrolar o ciclo o número de vezes que achar necessárias.

```python
assume n > 0 and n < 4;
x = 1; 
r = 1; 
i = 1;
if(i < n):
    x = x+2;
    r = r+x;
    i = i+1;
    if (i < n):
      x = x+2;
      r = r+x;
      i = i+1;
      assert not(i<n);
assert r == n*n;  
``` 

*b)* Escreve uma versão SA do programa.

```python
assume n0 > 0 and n0 < 4;
x1 = 1; 
r1 = 1; 
i1 = 1;
if(i1 < n0):
  x2 = x1+2;
  r2 = r1+x2;
  i2 = i1+1;
  if (i2 < n0):
    x3 = x2+2;
    r3 = r2+x3;
    i3 = i2+1;
    assert not(i3<n0);
    else:
      x3 = x2;
      r3 = r2;
      i3 = i2;
else:
  x3 = x1;
  r3 = r1;
  i3 = i1;
assert r3 == n0*n0;  
``` 

*c)* Escreva a tradução desse programa sem ciclos na linguagem de fluxos. 

 ```python
assume n0 > 0 and n0 < 4;
x1 = 1; r1 = 1; i1 = 1;
( assume (i1 < n0); x2 = x1+2; r2 = r1+x2; i2 = i1+1;
  ( assume(i2<n0); x3 = x2+2; r3 = r2+x3; i3 = i2+1; assert not(i3<n0); 
    ||
    assume not(i2<no); x3 = x2; r3 = r2; i3 = i2;
  )
  || 
  assume not(i1<no); x3=x1; r3=r1; i3=r1;
)
assert r3 == n0*n0;
``` 


*d)* Para simplificar a verificação, calcule o conjunto de fluxos sem escolha não-determinista que corresponde a este programa.

 ```python
assume n0 > 0 and n0 < 4;
x1 = 1; r1 = 1; i1 = 1;
assume (i1 < n0); x2 = x1+2; r2 = r1+x2; i2 = i1+1;
assume(i2<n0); x3 = x2+2; r3 = r2+x3; i3 = i2+1; assert not(i3<n0); 
assert r3 == n0*n0;
``` 

 ```python
assume n0 > 0 and n0 < 4;
x1 = 1; r1 = 1; i1 = 1;
assume (i1 < n0); x2 = x1+2; r2 = r1+x2; i2 = i1+1;
assume not(i2<no); x3 = x2; r3 = r2; i3 = i2;
assert r3 == n0*n0;
``` 

 ```python
assume n0 > 0 and n0 < 4;
x1 = 1; r1 = 1; i1 = 1;
assume not(i1<no); x3=x1; r3=r1; i3=r1;
assert r3 == n0*n0;
``` 
*e)* Para cada um destes fluxos, calcule a respectiva denotação lógica (VC). 

```python
[assume n0 > 0 and n0 < 4;
x1 = 1; r1 = 1; i1 = 1;
assume (i1 < n0); x2 = x1+2; r2 = r1+x2; i2 = i1+1;
assume(i2<n0); x3 = x2+2; r3 = r2+x3; i3 = i2+1; assert not(i3<n0); 
assert r3 == n0*n0;
]
=
[assume n0 > 0 and n0 < 4;
x1 = 1; r1 = 1; i1 = 1;
assume (i1 < n0); x2 = x1+2; r2 = r1+x2; i2 = i1+1;
assume(i2<n0); x3 = x2+2; r3 = r2+x3; i3 = i2+1;  assert not(i3<n0) and r3 == n0*n0;
]
=
[assume n0 > 0 and n0 < 4;
x1 = 1; r1 = 1; i1 = 1;
assume (i1 < n0); x2 = x1+2; r2 = r1+x2; i2 = i1+1;
assume(i2<n0); x3 = x2+2; r3 = r2+x3; i3 = i2+1;] => not(i3<n0) and r3 == n0*n0;
=
n0 > 0 and n0 < 4 and x1 == 1 and r1 == 1 and i1 == 1 and (i1 < n0) and x2 == x1+2 and r2 == r1+x2 and i2 == i1+1 and (i2<n0) and x3 == x2+2 and r3 == r2+x3 and i3 == i2+1 => 
not(i3<n0) and r3 == n0*n0
``` 

 ```python
[assume n0 > 0 and n0 < 4;
x1 = 1; r1 = 1; i1 = 1;
assume (i1 < n0); x2 = x1+2; r2 = r1+x2; i2 = i1+1;
assume not(i2<no); x3 = x2; r3 = r2; i3 = i2;
assert r3 == n0*n0;
]
=
n0 > 0 and n0 < 4  and 
x1 == 1 and  r1 == 1 and i1 == 1 and 
(i1 < n0) and x2 == x1+2 and r2 == r1+x2 and 
i2 == i1+1 and not(i2<no) and x3 == x2 
and r3 == r2 and i3 == i2 => r3 == n0*n0
``` 

 ```python
[assume n0 > 0 and n0 < 4;
x1 = 1; r1 = 1; i1 = 1;
assume not(i1<no); x3=x1; r3=r1; i3=r1;
assert r3 == n0*n0;
]
=
n0 > 0 and n0 < 4 and x1 == 1 and r1 == 1 and i1 == 1 and not(i1<no) and x3==x1 and r3==r1 and i3==r1 => r3 == n0*n0
``` 

*f)* Use o Z3 para provar as VCs que calculou.

In [ ]:
n0, x1, x2, x3, r1, r2, r3, i1, i2, i3 = Ints('n0, x1, x2, x3, r1, r2, r3, i1, i2, i3')

cond1 = Implies(And(n0>0,n0<4, x1==1, r1==1, i1==1, i1<n0, x2==x1+2, r2==r1+x2, i2==i1+1, i2<n0, x3==x2+2, r3==r2+x3, i2==i2+1) ,And(Not(i3<n0),r3==n0*n0))

prove(cond1)

cond2 = Implies(And(n0>0,n0<4, x1==1, r1==1, i1==1, i1<n0, x2==x1+2, r2==r1+x2, i2==i1+1, Not(i2<n0), x3==x2, r3==r2, i3==i2),r3==n0*n0)

prove(cond2)

cond3 = Implies(And(n0>0,n0<4, x1==1, r1==1, i1==1, Not(i1<n0), x3==x1, r3==r1, i3==r1),r3==n0*n0)

prove(cond3)

Proved
Proved
Proved


### Exercício 3
Considere agora que a pré-condição do programa anterior é simplificada para `assume n > 0`, e aplique a técnica do desenrolamento dos ciclos. Exprimente tanto com a *unwinding assertion* como com a *unwinding assumption*. 

Pronuncie-se quanto à correcção deste programa, e comente os resultados obtidos.

In [ ]:
n0, x1, x2, x3, r1, r2, r3, i1, i2, i3 = Ints('n0, x1, x2, x3, r1, r2, r3, i1, i2, i3')

cond1 = Implies(And(n0>0, x1==1, r1==1, i1==1, i1<n0, x2==x1+2, r2==r1+x2, i2==i1+1, i2<n0, x3==x2+2, r3==r2+x3, i2==i2+1), And(Not(i3<n0),r3==n0*n0))

prove(cond1)

cond2 = Implies(And(n0>0, x1==1, r1==1, i1==1, i1<n0, x2==x1+2, r2==r1+x2, i2==i1+1, Not(i2<n0), x3==x2, r3==r2, i3==i2),r3==n0*n0)

prove(cond2)

cond3 = Implies(And(n0>0, x1==1, r1==1, i1==1, Not(i1<n0), x3==x1, r3==r1, i3==r1),r3==n0*n0)

prove(cond3)

Proved
Proved
Proved


## Codificação de funções lógicas

### Exercício 4

Considere o seguinte programa `FACT` para o cálculo do factorial de um número:

```python
f = 1; i = 1;
while (i <= n):
    f = f*i;
    i = i+1;
```

Apresente uma especificação adequada para este programa incluindo um invariante de ciclo que permita provar a sua correção.

```python
f = 1; i = 1;
while (i <= n):
    invariante: 0<i and i<=n+1 and f == (i-1)!
    f = f*i;
    i = i+1;
```




Na especificação do programa anterior recorreu certamente à função matemática $(n)!$ que denota o factorial de $n$.
Para poder usar um *SMT solver* para o ajudar a verificar a correção do programa face à sua especificação, terá acrescentar à teoria do *SMT solver* a codificação lógica da função factorial, uma vez que a teoria de inteiros não vem munida de tal função.

Uma forma de o fazer é definir na teoria de inteiros uma função lógica ${\sf fact}$ cuja semântica é dada por um conjunto de axiomas. Por exemplo, assim:

$$
\begin{array}{l}
{\sf fact} (0) = 1 \\
\forall n . \, n > 0 \to {\sf fact}(n) = n * {\sf fact}(n-1)
\end{array}
$$

### Exercício 5

Codifique a função factorial no Z3, de acordo com a axiomatização apresentada acima. Consulte o manual do Z3 em https://www.cs.tau.ac.il/~msagiv/courses/asv/z3py/guide-examples.htm para saber como declarar e utilizar funções lógicas. Teste a correção da sua axiomatização verificando que o valor de $\mathsf{fact}(5) = 120$.

In [ ]:
fact = Function('fact',IntSort(),IntSort())
n = Int('n')
axiomas = And(fact(0)==1, ForAll(n, Implies(n>0,fact(n)==n*fact(n-1))))
prove(Implies(axiomas, fact(5)==120))

Proved


### Exercício 6

Aplique agora a metodologia *havoc* para provar a correcção do programa `FACT` face à especificação que escreveu.